In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from sandbox import rrtiris

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
import meshcat
from pydrake.all import GenerateSeedingPolytope
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower, FindEpsilonLowerVector, FindEpsilonUpperVector
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import FindMaxEpsForAllIneqs, MosekSolver, MosekSolverDetails


# Build Plant


In [4]:
q0 = [0.0, 0.0, 0.0]
q_low  = [-1.7, -2., -1.7]
q_high = [ 1.7,  2.,  1.7]


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/twoDOF_iiwa7_with_box_collision.sdf")

In [6]:
box_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



locs = [[0.,0.,0.],[0.,.5,0.],[0.,-.5,0.]]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", models[0]), RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
            
#q_low = plant.GetPositionLowerLimits().tolist()
#q_high = plant.GetPositionUpperLimits().tolist()

Ratfk = RationalForwardKinematics(plant)


In [7]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram
visualizer.visualize_collision_constraint(N = 30)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [8]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [9]:
#filter fused joints self collisions 
digaram = visualizer.diagram
context = visualizer.diagram_context
plant_context = visualizer.plant_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
# print([get_name_of_gid(gid) for gid in oneDOF_fused_col_geom])
# print([get_name_of_gid(gid) for gid in twoDOF_fused_col_geom])
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

150 -> 150
137 -> 150


# Setup IRIS Options and Generate Regions

In [10]:
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.0, -1.3, -1.3],  # start
#                         [0.9, -1.5, -0.8]     # goal
#                          ])
seed_points_q = np.array([[0.0, 0, 0], # zero config
                        [0.8, 1.3, -0.8],  # START: blue low green up
                        [0.1, 0.9, -1.2],     # GOAL: green low other up
                        [0.2, 1.6, -0.6],
                        [0.5, -1.9, -0.9]
                         ])[:,(0,1,2)]   # passing


seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)

start = seed_points[1,:]
goal = seed_points[2,:]

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), np.zeros((3,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])
context = diagram.CreateDefaultContext()
q_star = np.zeros(3)
def check_empty_polytope(C, d):
    prog = MathematicalProgram()
    t = prog.NewContinuousVariables(C.shape[1], "t")

    prog.AddLinearConstraint(C,-np.inf*np.ones(C.shape[0]), d, t)
    prog.AddBoundingBoxConstraint(limits_t[0], limits_t[1], t)
    result = MosekSolver().Solve(prog)
    return result.is_success(), result.get_solution_result()

In [11]:
# RUN THIS CELL IF YOU WANT TO DO SNOPT IRIS

# regions = []

# iris_options = IrisOptionsRationalSpace()
# iris_options.require_sample_point_is_contained = True
# iris_options.iteration_limit = 20
# iris_options.configuration_space_margin = 1e-5
# iris_options.max_faces_per_collision_pair = 60
# iris_options.termination_threshold = -1
# iris_options.q_star = np.zeros(3)
# iris_options.relative_termination_threshold = 0.05
# iris_options.enable_ibex = False
# #deprecated
# iris_options.certify_region_with_sos_during_generation = False
# iris_options.certify_region_with_sos_after_generation = False

# for i, s in enumerate(seed_points):
#     plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
#     if False:
#         #starting_hpolyhedron = regions[i-1]
#         r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
#                                               iris_options, starting_hpolyhedron)
#     else:
#         r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
#     regions.append(r)
#     print(f'Completed region: {i+1}/{len(seed_points)}')
#     print(f"Sample point contained = {np.all(r.A()@s <= r.b())}")
#     print(f"Sample point contained componentwise = {r.A()@s <= r.b()}")
#     print()

In [12]:
# if do_viz:
#     visualizer.plot_regions(regions, ellipses=None, region_suffix='original')

## Create default region

In [38]:
regions_default = []
max_vects = []
min_vects = []
vector_bisection_options_list = []
bilinear_alteration_options_list = []
interleaved_options_list = []

#parameters for creating a default region. Currently set to create a box
num_perm_dim = 0
num_rot = 1

for i, s in enumerate(seed_points):
    regions_default.append(GenerateSeedingPolytope(s, num_perm_dim, num_rot))
    
    eps_min_vect = FindEpsilonLowerVector(regions_default[i].A(), 
                                      regions_default[i].b(), 
                                      limits_t[0], limits_t[1], s)
    
    min_vects.append(eps_min_vect)
    
    eps_max_vect = FindMaxEpsForAllIneqs(plant, plant.GetMyMutableContextFromRoot(context), 
                                q_star, regions_default[i].A(), regions_default[i].b(), 
                                         eps_min_vect, limits_t[0], limits_t[1], s)
    max_vects.append(eps_max_vect)
    
    vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
    vector_bisection_search_options.max_iters = 10
    vector_bisection_search_options.max_feasible_iters = 5
    vector_bisection_search_options.search_d = True
    vector_bisection_search_options.epsilon_min = eps_min_vect
    vector_bisection_search_options.epsilon_max = eps_max_vect
    vector_bisection_search_options.num_threads = -1
    vector_bisection_options_list.append(vector_bisection_search_options)
    
    bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
    bilinear_alternation_options.max_iters = 20
    bilinear_alternation_options.lagrangian_backoff_scale = 0
    bilinear_alternation_options.polytope_backoff_scale = 0
    bilinear_alternation_options.num_threads = -1
    bilinear_alteration_options_list.append(bilinear_alternation_options)
    
    interleaved_options = rational_forward_kinematics.InterleavedRegionSearchOptions()
    interleaved_options.vector_bisection_search_options = vector_bisection_search_options
    interleaved_options.bilinear_alternation_options = bilinear_alternation_options
    #actually does round robin
    interleaved_options.use_vector_bisection_search = True
    interleaved_options.max_method_switch = 3
    interleaved_options_list.append(interleaved_options)
    
    
num_round_robin_rounds = 1

if do_viz:
    visualizer.plot_regions(regions_default, ellipses=None, region_suffix='_default')
    

In [39]:
i = 2
s = seed_points[i,:]
C, d = regions_default[i].A(), regions_default[i].b()
vector_bisection_search_options = vector_bisection_options_list[i]
interleaved_options = interleaved_options_list[i]

In [40]:
C.shape

(6, 3)

In [41]:
print(vector_bisection_search_options.epsilon_max)
print(vector_bisection_search_options.epsilon_min)
print(vector_bisection_search_options.epsilon_max-
     vector_bisection_search_options.epsilon_min)

[ 0.05004171 -0.09999999  0.73417852  0.05004171  0.53309677 -0.10000001]
[-0.09999999 -0.09999999 -0.09999999 -0.10000001 -0.10000001 -0.10000001]
[ 1.50041698e-01 -4.44089210e-16  8.34178507e-01  1.50041718e-01
  6.33096784e-01  5.55111512e-17]


In [42]:
# #plot the new polytopes which each individual face moved
# if do_viz:
#     for i, r in enumerate(regions_default):
#         C, d = r.A(), r.b()
#         eps_max = vector_bisection_options_list[i].epsilon_max
#         r_moved_list = []
#         for j in range(len(d)):
#             d_new = d.copy()
#             d_new[j] += eps_max[j]
#             r_moved_list.append(HPolyhedron(C, d_new))
#         if do_viz:
#             visualizer.plot_regions(r_moved_list, ellipses=None, region_suffix=f'_default_{i}_maxed')
#         if do_viz:
#             visualizer.plot_regions([HPolyhedron(C, d+0.99*eps_max)], ellipses=None, region_suffix=f'_all_{i}_maxed')

## Run Cspace Free Region

In [43]:
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
# solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

In [44]:
cspace_free_region_solution_interleaved = cspace_free_region.InterleavedCSpacePolytopeSearch(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, num_round_robin_rounds, 
                                                                     interleaved_options,
                                                                     solver_options, s)

[2022-02-08 23:25:57.057] [console] [info] Starting bisections
[2022-02-08 23:25:59.604] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:26:00.873] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:26:00.889] [console] [info] Ineq 1/6 in round 1
[2022-02-08 23:26:00.890] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:26:00.896] [console] [info] output of eps_redundant_max is 1.8224695040938002
[2022-02-08 23:26:00.897] [console] [info] current gap is epsilon_gap=0.8341784992408996
[2022-02-08 23:26:02.776] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:26:04.011] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:26:04.029] [console] [info] epsilon=0.3170892670967813 is feasible
[2022-02-08 23:26:04.030] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:26:04.037] 

[2022-02-08 23:26:35.719] [console] [info] epsilon=-0.024665965388246513 is infeasible
[2022-02-08 23:26:35.720] [console] [info] current gap is epsilon_gap=0.0747076737637853
[2022-02-08 23:26:37.581] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@0000000004700bd8, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)

[2022-02-08 23:26:37.582] [console] [info] epsilon=-0.06201980227013917 is infeasible
[2022-02-08 23:26:37.583] [console] [info] current gap is epsilon_gap=0.03735383688189265
[2022-02-08 23:26:39.407] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:26:40.640] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:26:40.670] [console] [info] epsilon=-0.0806967207110855 is feasible
[2022-02-08 23:26:40.671] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:26:40.677] [console] [info] output of eps_redundant_max is 1.

[2022-02-08 23:27:07.519] [console] [info] current gap is epsilon_gap=0.016410799877457485
[2022-02-08 23:27:09.299] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_3_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-08 23:27:09.300] [console] [info] epsilon=-0.07303929070539242 is infeasible
[2022-02-08 23:27:09.300] [console] [info] current gap is epsilon_gap=0.00820539993872875
[2022-02-08 23:27:11.117] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:27:12.479] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:27:12.503] [console] [info] epsilon=-0.0771419906747568 is feasible
[2022-02-08 23:27:12.503] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:27:12.516] [console] [info] output of eps_redundant_max i

[2022-02-08 23:27:51.617] [console] [info] cost improvement 0.0011191400470564328
[2022-02-08 23:27:52.926] [console] [info] Iter: 10, polytope step cost -0.004940738339393831, solver time 0.31127500534057617
[2022-02-08 23:27:54.742] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:27:54.743] [console] [info] Lagrangian step time 1.797 s
[2022-02-08 23:27:54.754] [console] [info] max(power(det(P), 1/4))=0.2530851320398552, solver_time 0.004654884338378906
[2022-02-08 23:27:54.755] [console] [info] cost improvement 0.001076334570370041
[2022-02-08 23:27:56.234] [console] [info] Iter: 11, polytope step cost -0.004803616059098791, solver time 0.49167895317077637
[2022-02-08 23:27:58.049] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:27:58.050] [console] [info] Lagrangian step time 1.788 s
[2022-02-08 23:27:58.063] [console] [info] max(power(det(P), 1/4))=0.25411964644636853, solver_time 0.00743412971496582
[2022-02-08 2

[2022-02-08 23:28:32.041] [console] [info] epsilon=0.0076288078184245205 is feasible
[2022-02-08 23:28:32.042] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:28:32.049] [console] [info] output of eps_redundant_max is 0.9183393805397282
[2022-02-08 23:28:32.049] [console] [info] reset eps_min=0.008302824655234214, eps_max=0.7341785167172311
[2022-02-08 23:28:32.049] [console] [info] Ineq 2/6 in round 1
[2022-02-08 23:28:32.076] [console] [info] output of eps_redundant_max is 2.032412028644645
[2022-02-08 23:28:32.076] [console] [info] current gap is epsilon_gap=1.2091319234011464
[2022-02-08 23:28:33.873] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:28:35.150] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:28:35.170] [console] [info] epsilon=-0.07146918770845595 is feasible
[2022-02-08 23:28:35.171] [console] [info] epsilon exceeds components of epsilon max in some com

...

[2022-02-08 23:30:13.714] [console] [info] cost improvement 0.001301405001834649
[2022-02-08 23:30:15.002] [console] [info] Iter: 10, polytope step cost -0.0045132248431117164, solver time 0.2938539981842041
[2022-02-08 23:30:16.915] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:30:16.915] [console] [info] Lagrangian step time 1.892 s
[2022-02-08 23:30:16.925] [console] [info] max(power(det(P), 1/4))=0.24484604185323705, solver_time 0.004207134246826172
[2022-02-08 23:30:16.926] [console] [info] cost improvement 0.0012401985480764155
[2022-02-08 23:30:18.331] [console] [info] Iter: 11, polytope step cost -0.004357335019722837, solver time 0.41805291175842285
[2022-02-08 23:30:20.098] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:30:20.099] [console] [info] Lagrangian step time 1.744 s
[2022-02-08 23:30:20.109] [console] [info] max(power(det(P), 1/4))=0.24602923839147128, solver_time 0.004553079605102539
[2022-02-08

[2022-02-08 23:30:55.497] [console] [info] epsilon=0.07076882291855764 is infeasible
[2022-02-08 23:30:55.497] [console] [info] current gap is epsilon_gap=0.09477281339981049
[2022-02-08 23:30:57.360] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box4_Collision@00000000046fbfd8, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(box_scene::box4_Collision@00000000046fbfd8, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-08 23:30:57.361] [console] [info] epsilon=0.023382416218652398 is infeasible
[2022-02-08 23:30:57.361] [console] [info] current gap is epsilon_gap=0.047386406699905244
[2022-02-08 23:30:59.304] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:31:00.523] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:31:00.551] [console] [info] epsilon=-0.00031078713130022403 is feasible
[2022-02-08 23:31:00.551] [console] [info] epsilon exceeds components of epsilon max in some components, increa

[2022-02-08 23:31:21.608] [console] [info] epsilon=0.2719897283358421 is infeasible
[2022-02-08 23:31:21.608] [console] [info] current gap is epsilon_gap=0.261107045656275
[2022-02-08 23:31:23.665] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_7_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_7_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-08 23:31:23.666] [console] [info] epsilon=0.1414362055077046 is infeasible
[2022-02-08 2

[2022-02-08 23:31:54.127] [console] [info] epsilon=0.0029398207432379817 is infeasible
[2022-02-08 23:31:54.127] [console] [info] current gap is epsilon_gap=0.003140125842153383
[2022-02-08 23:31:56.058] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-08 23:31:56.059] [console] [info] epsilon=0.00136975782216129 is infeasible
[2022-02-08 23:31:56.059] [console] [info] current gap is epsilon_gap=0.0015700629210766916
[2022-02-08 23:31:58.019] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:31:59.253] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:31:59.282] [console] [info] epsilon=0.0005847263616229443 is feasible
[2022-02-08 23:31:59.283] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:31:59.292] [console] [info] output of eps_redundant_max 

[2022-02-08 23:32:38.592] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:32:38.593] [console] [info] Lagrangian step time 1.783 s
[2022-02-08 23:32:38.606] [console] [info] max(power(det(P), 1/4))=0.24265839905056003, solver_time 0.006105899810791016
[2022-02-08 23:32:38.607] [console] [info] cost improvement 0.0013698879027291
[2022-02-08 23:32:40.036] [console] [info] Iter: 9, polytope step cost -0.004714635077148964, solver time 0.42795300483703613
[2022-02-08 23:32:41.779] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:32:41.780] [console] [info] Lagrangian step time 1.715 s
[2022-02-08 23:32:41.792] [console] [info] max(power(det(P), 1/4))=0.24397185036947955, solver_time 0.00545501708984375
[2022-02-08 23:32:41.792] [console] [info] cost improvement 0.001313451318919523
[2022-02-08 23:32:43.132] [console] [info] Iter: 10, polytope step cost -0.004533380425126114, solver time 0.36126279830932617
[2022-02-08 23:3

In [45]:
r = HPolyhedron(np.vstack([starting_poly.A(),cspace_free_region_solution_interleaved.C]),
                np.hstack([starting_poly.b(), cspace_free_region_solution_interleaved.d])
                )
e = Hyperellipsoid(np.linalg.inv(cspace_free_region_solution_interleaved.P), 
                   cspace_free_region_solution_interleaved.q)
if do_viz:
    visualizer.plot_regions([r], 
                            ellipses=[e], region_suffix='_interleaved_single')

In [23]:
r = HPolyhedron(np.vstack([starting_poly.A(),cspace_free_region_solution_round_robin.C]),
                np.hstack([starting_poly.b(), cspace_free_region_solution_round_robin.d])
                )
e = Hyperellipsoid(np.linalg.inv(cspace_free_region_solution_round_robin.P), 
                   cspace_free_region_solution_round_robin.q)
if do_viz:
    visualizer.plot_regions([r], 
                            ellipses=[e], region_suffix='_round_robin_single')

In [47]:
seed_point_list = [s for s in seed_points] 

C_mats = [r.A() for r in regions_default]
d_vects = [r.b() for r in regions_default]
cspace_free_region_solution_interleaved_vect = cspace_free_region.InterleavedCSpacePolytopeSearchForSeedPoints(
                                                                    q_star,
                                                                     filtered_collision_pairs,
                                                                     C_mats, d_vects, num_round_robin_rounds, 
                                                                     interleaved_options_list,
                                                                     solver_options, seed_point_list)
# cspace_free_region_solution_round_robin_vect = cspace_free_region.CspacePolytopeRoundRobinBisectionSearchForSeedPoints(
#                                                                     q_star,
#                                                                      filtered_collision_pairs,
#                                                                      C_mats, d_vects, num_round_robin_rounds, 
#                                                                      vector_bisection_options_list,
#                                                                      solver_options, seed_point_list)

[2022-02-08 23:34:42.168] [console] [info] Starting bisections
[2022-02-08 23:34:42.168] [console] [info] Starting bisections
[2022-02-08 23:34:42.168] [console] [info] Starting bisections
[2022-02-08 23:34:42.168] [console] [info] Starting bisections
[2022-02-08 23:34:42.168] [console] [info] Starting bisections
[2022-02-08 23:34:55.647] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:34:55.702] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:34:55.716] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:34:55.716] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:34:55.716] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:34:57.175] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:34:57.204] [console] [info] Ineq 1/6 in round 1
[2022-02-08 23:34:57.205] [console] [info] epsilon exceeds components of epsilon max i

[2022-02-08 23:35:20.652] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:35:20.980] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:35:21.796] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:35:21.827] [console] [info] epsilon=0.060374711779954876 is feasible
[2022-02-08 23:35:21.828] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:35:21.829] [console] [info] output of eps_redundant_max is 2.15710556183969
[2022-02-08 23:35:21.830] [console] [info] reset eps_min=0.06050656194888848, eps_max=1.182997617871838
[2022-02-08 23:35:21.830] [console] [info] current gap is epsilon_gap=1.1224910559229495
[2022-02-08 23:35:23.889] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:35:23.932] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDO

[2022-02-08 23:35:44.430] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:35:45.023] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:35:45.045] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:35:45.212] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:35:45.240] [console] [info] epsilon=0.9992913918351234 is feasible
[2022-02-08 23:35:45.240] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:35:45.242] [console] [info] output of eps_redundant_max is 1.4758901668235191
[2022-02-08 23:35:45.242] [console] [info] reset eps_min=1.0111561148817474, eps_max=1.1299732291358144
[2022-02-08 23:35:45.242] [console] [info] current gap is epsilon_gap=0.11881711425406705
[2022-02-08 23:35:45.624] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:35:46.704] [console] [info] epsilon=-0.007676731546895271 is feasib

[2022-02-08 23:36:06.488] [console] [info] epsilon=0.9833669369921425 is infeasible
[2022-02-08 23:36:06.488] [console] [info] current gap is epsilon_gap=0.6703575735165926
[2022-02-08 23:36:08.034] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:36:08.073] [console] [info] epsilon=0.7198326997416092 is feasible
[2022-02-08 23:36:08.074] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:36:08.093] [console] [info] output of eps_redundant_max is 0.9962445849153404
[2022-02-08 23:36:08.094] [console] [info] reset eps_min=0.7262249366547913, eps_max=0.7341785167172311
[2022-02-08 23:36:08.094] [console] [info] current gap is epsilon_gap=0.007953580062439847
[2022-02-08 23:36:10.578] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:36:10.662] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF:

[2022-02-08 23:36:29.323] [console] [info] current gap is epsilon_gap=0.32958029011518053
[2022-02-08 23:36:31.094] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:36:31.138] [console] [info] epsilon=1.1194485369565481 is feasible
[2022-02-08 23:36:31.149] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:36:31.151] [console] [info] output of eps_redundant_max is 1.363481318741142
[2022-02-08 23:36:31.154] [console] [info] reset eps_min=1.1235649629641242, eps_max=1.1299732291358144
[2022-02-08 23:36:31.155] [console] [info] current gap is epsilon_gap=0.006408266171690213
[2022-02-08 23:36:31.960] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box4_Collision@00000000046fbfd8, iiwa7_oneDOF::iiwa_oneDOF_link_5_collision)
(box_scene::box4_Collision@00000000046fbfd8, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(box_scene::box4_Collision@00000000046fbfd8, iiwa7_oneDOF

...

[2022-02-08 23:57:50.510] [console] [info] Ineq 1/6 in round 1
[2022-02-08 23:57:50.512] [console] [info] output of eps_redundant_max is 0.23713082257603202
[2022-02-08 23:57:50.512] [console] [info] current gap is epsilon_gap=0.23713082257603202
[2022-02-08 23:57:52.342] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:57:52.485] [console] [info] Iter: 9, polytope step cost -0.004714635077148964, solver time 2.1843512058258057
[2022-02-08 23:57:54.439] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:57:54.440] [console] [info] Lagrangian step time 1.935 s
[2022-02-08 23:57:54.452] [console] [info] max(power(det(P), 1/4))=0.24397185036947955, solver_time 0.00529789924621582
[2022-02-08 23:57:54.453] [console] [info] cost improvement 0.001313451318919523
[2022-02-08 23:57:55.341] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:57:55.359] [console] [info] epsilon=-1.738967281700706 is feasible
[2022-02-08 23:5

[2022-02-08 23:58:28.383] [console] [info] reset eps_min=-0.024460843388278253, eps_max=-2.7755575615628914e-17
[2022-02-08 23:58:28.384] [console] [info] current gap is epsilon_gap=0.024460843388278225
[2022-02-08 23:58:30.379] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:58:31.160] [console] [info] Iter: 16, polytope step cost -0.003607621190353328, solver time 0.3836789131164551
[2022-02-08 23:58:31.201] [console] [info] max(power(det(P), 1/4))=0.25160738904047164, solver_time 0.019369840621948242
[2022-02-08 23:58:31.646] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:58:31.663] [console] [info] epsilon=-0.01223042169413914 is feasible
[2022-02-08 23:58:31.664] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:58:31.671] [console] [info] output of eps_redundant_max is 1.1601161764267833
[2022-02-08 23:58:31.671] [console] [info] reset eps_min=-0.008783024576258791, e

[2022-02-08 23:59:05.318] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:59:05.326] [console] [info] output of eps_redundant_max is 0.8866093334687197
[2022-02-08 23:59:05.326] [console] [info] reset eps_min=-0.0004867506063769844, eps_max=-4.163336342344337e-17
[2022-02-08 23:59:05.326] [console] [info] current gap is epsilon_gap=0.0004867506063769428
[2022-02-08 23:59:07.112] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:59:08.290] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:59:08.305] [console] [info] epsilon=-0.00024337530318851303 is feasible
[2022-02-08 23:59:08.306] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:59:08.313] [console] [info] output of eps_redundant_max is 0.8863658827944811
[2022-02-08 23:59:08.313] [console] [info] reset eps_min=-0.0002432999320635676, eps_max=-4.16333634234

[2022-02-08 23:59:42.797] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:59:42.805] [console] [info] output of eps_redundant_max is 0.9767780956650265
[2022-02-08 23:59:42.805] [console] [info] reset eps_min=-0.03913215797398145, eps_max=-2.7755575615628914e-17
[2022-02-08 23:59:42.805] [console] [info] current gap is epsilon_gap=0.039132157973981424
[2022-02-08 23:59:44.584] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 23:59:45.786] [console] [info] bilinear alt on d succeeded
[2022-02-08 23:59:45.803] [console] [info] epsilon=-0.01956607898699074 is feasible
[2022-02-08 23:59:45.804] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 23:59:45.810] [console] [info] output of eps_redundant_max is 0.9571464603652513
[2022-02-08 23:59:45.811] [console] [info] reset eps_min=-0.019500522671887288, eps_max=-2.7755575615628914e

[2022-02-09 00:00:19.155] [console] [info] epsilon=-0.01558490143322916 is infeasible
[2022-02-09 00:00:19.155] [console] [info] current gap is epsilon_gap=0.00519496714440972
[2022-02-09 00:00:20.984] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@0000000004700bd8, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)

[2022-02-09 00:00:20.985] [console] [info] epsilon=-0.01818238500543402 is infeasible
[2022-02-09 00:00:20.986] [console] [info] current gap is epsilon_gap=0.002597483572204861
[2022-02-09 00:00:22.843] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@0000000004700bd8, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)

[2022-02-09 00:00:22.844] [console] [info] epsilon=-0.01948112679153645 is infeasible
[2022-02-09 00:00:22.844] [console] [info] current gap is epsilon_gap=0.0012987417861024322
[2022-02-09 00:00:24.843] [console] [info] Found Lagrangian multiplier and 

In [48]:
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_interleaved_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=ellipses_round_robin, region_suffix='_interleaved')

# Don't run after here

In [22]:
cspace_free_region_solution_round_robin = cspace_free_region.CspacePolytopeRoundRobinBisectionSearch(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, num_rounds, 
                                                                     vector_bisection_search_options,
                                                                     solver_options, s)

[2022-02-08 19:29:47.629] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:29:49.330] [console] [info] bilinear alt on d succeeded
[2022-02-08 19:29:49.358] [console] [info] Ineq 1/6 in round 1
[2022-02-08 19:29:49.359] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 19:29:49.379] [console] [info] output of eps_redundant_max is 1.8224695040938002
[2022-02-08 19:29:49.381] [console] [info] current gap is epsilon_gap=0.8341784992408996
[2022-02-08 19:29:52.187] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:29:53.997] [console] [info] bilinear alt on d succeeded
[2022-02-08 19:29:54.033] [console] [info] epsilon=0.3170892670967813 is feasible
[2022-02-08 19:29:54.034] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 19:29:54.051] [console] [info] output of eps_redundant_max is 1.3733133612699

[2022-02-08 19:30:42.509] [console] [info] epsilon=-0.024665965387383592 is infeasible
[2022-02-08 19:30:42.510] [console] [info] current gap is epsilon_gap=0.07470767376292238
[2022-02-08 19:30:45.547] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@00005642fcbecef8, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)

[2022-02-08 19:30:45.549] [console] [info] epsilon=-0.062019802268844786 is infeasible
[2022-02-08 19:30:45.549] [console] [info] current gap is epsilon_gap=0.03735383688146119
[2022-02-08 19:30:48.945] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:30:50.770] [console] [info] bilinear alt on d succeeded
[2022-02-08 19:30:50.810] [console] [info] epsilon=-0.08069672070957538 is feasible
[2022-02-08 19:30:50.811] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 19:30:50.821] [console] [info] output of eps_redundant_max is

[2022-02-08 19:31:36.939] [console] [info] current gap is epsilon_gap=0.01641079987684163
[2022-02-08 19:31:40.455] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_3_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-08 19:31:40.457] [console] [info] epsilon=-0.07303929070077358 is infeasible
[2022-02-08 19:31:40.458] [console] [info] current gap is epsilon_gap=0.008205399938420815
[2022-02-08 19:31:43.692] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:31:45.558] [console] [info] bilinear alt on d succeeded
[2022-02-08 19:31:45.600] [console] [info] epsilon=-0.07714199066998399 is feasible
[2022-02-08 19:31:45.601] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 19:31:45.627] [console] [info] output of eps_redundant_max 

In [29]:
cspace_free_region_solution_bilinear = cspace_free_region.CspacePolytopeBilinearAlternation(q_star,
                                                                     filtered_collision_pairs,
                                                                     cspace_free_region_solution_round_robin.C,
                                                                    0.75*cspace_free_region_solution_round_robin.d,
                                                                     bilinear_alternation_options,
                                                                     solver_options, s)

[2022-02-08 19:34:36.721] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:34:36.723] [console] [info] Lagrangian step time 3.045 s
[2022-02-08 19:34:36.751] [console] [info] max(power(det(P), 1/4))=0.2071381570004865, solver_time 0.018611907958984375
[2022-02-08 19:34:36.753] [console] [info] cost improvement inf
[2022-02-08 19:34:38.615] [console] [info] Iter: 0, polytope step cost -0.008828768121858371, solver time 0.48531293869018555
[2022-02-08 19:34:41.859] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:34:41.860] [console] [info] Lagrangian step time 3.218 s
[2022-02-08 19:34:41.881] [console] [info] max(power(det(P), 1/4))=0.2105033555416147, solver_time 0.013540029525756836
[2022-02-08 19:34:41.882] [console] [info] cost improvement 0.0033651985411282115
[2022-02-08 19:34:43.780] [console] [info] Iter: 1, polytope step cost -0.008522369595631428, solver time 0.532606840133667
[2022-02-08 19:34:46.920] [console

[2022-02-08 19:36:00.640] [console] [info] Iter: 16, polytope step cost -0.00557567747292513, solver time 0.38878607749938965
[2022-02-08 19:36:04.137] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:36:04.141] [console] [info] Lagrangian step time 3.472 s
[2022-02-08 19:36:04.172] [console] [info] max(power(det(P), 1/4))=0.24350116763464993, solver_time 0.01593494415283203
[2022-02-08 19:36:04.174] [console] [info] cost improvement 0.001400599503426303
[2022-02-08 19:36:05.966] [console] [info] Iter: 17, polytope step cost -0.005466929346269455, solver time 0.3860509395599365
[2022-02-08 19:36:09.113] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:36:09.114] [console] [info] Lagrangian step time 3.119 s
[2022-02-08 19:36:09.142] [console] [info] max(power(det(P), 1/4))=0.24485582644352566, solver_time 0.00913095474243164
[2022-02-08 19:36:09.145] [console] [info] cost improvement 0.001354658808875725
[2022-02-08 19:3

In [31]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution_bilinear.C,
                                         cspace_free_region_solution_bilinear.d)], 
                            ellipses=None, region_suffix='_bilinear')

In [15]:
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_round_robin_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=ellipses_round_robin, region_suffix='_round_robin')

In [22]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution.C, 0.5*cspace_free_region_solution.d)], 
                            ellipses=None, region_suffix='_round_robin')

In [26]:
cspace_free_region_solution_vector_bisection = cspace_free_region.CspacePolytopeBisectionSearchVector(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, 
                                                                     vector_bisection_search_options,
                                                                     solver_options, s)

[2022-02-07 15:27:37.125] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-07 15:27:37.194] [console] [info] max(power(det(P), 1/4))=-nan, solver_time 0.05913114547729492
[2022-02-07 15:27:40.385] [console] [info] bilinear alt on d succeeded
[2022-02-07 15:27:40.411] [console] [info] max(power(det(P), 1/4))=1.195363453968912e-06, solver_time 0.014796972274780273
[2022-02-07 15:27:45.796] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_3_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-07 15:27:45.797] [console] [info] total iter=1, feasible iter = 0
[2022-02-07 15:27:51.001] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_

In [29]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution_vector_bisection.C, 
                                         cspace_free_region_solution_vector_bisection.d)], 
                            ellipses=None, region_suffix='_vector_bisection')

(True, <SolutionResult.kSolutionFound: 0>)


In [96]:
i

2

In [84]:
vector_bisection_search_options_list = []
scalar_bisection_search_options_list = []
bilinear_alternation_options_list = []
integrated_search_options_list = []


search_tuples = []

for i, r in enumerate(regions_default):
    vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
    vector_bisection_search_options.max_iters = 20
    vector_bisection_search_options.max_feasible_iters = 5
    vector_bisection_search_options.search_d = True
    vector_bisection_search_options.epsilon_min = FindEpsilonLowerVector(r.A(), r.b(),
                                                             limits_t[0], limits_t[1], seed_points[i,:])
    vector_bisection_search_options.epsilon_max = 0.25*FindEpsilonUpperVector(r.A(), 
                                      r.b(), 
                                      limits_t[0], limits_t[1])
    vector_bisection_search_options_list.append(vector_bisection_search_options)
    
    scalar_bisection_search_options = rational_forward_kinematics.BinarySearchOption()
    scalar_bisection_search_options.epsilon_max = 1
    scalar_bisection_search_options.max_iters = 15
    scalar_bisection_search_options.search_d = True
    scalar_bisection_search_options.epsilon_min = FindEpsilonLower(r.A(), r.b(),
                                                         limits_t[0], limits_t[1],
                                                         seed_points[i,:])
    scalar_bisection_search_options_list.append(scalar_bisection_search_options)
    
    
    bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
    bilinear_alternation_options.max_iters = 20
    bilinear_alternation_options.lagrangian_backoff_scale = 1e-6
    bilinear_alternation_options.polytope_backoff_scale = 1e-6
    bilinear_alternation_options_list.append(bilinear_alternation_options)
    
    integrated_search_options = rational_forward_kinematics.InterleavedRegionSearchOptions()
    integrated_search_options.vector_bisection_search_options = vector_bisection_search_options
    integrated_search_options.scalar_binary_search_options = scalar_bisection_search_options
    integrated_search_options.bilinear_alternation_options = bilinear_alternation_options
    integrated_search_options.max_method_switch = 2
    integrated_search_options.use_vector_bisection_search = True
    integrated_search_options_list.append(integrated_search_options)
    
    search_tuples.append((r, integrated_search_options, seed_points[i,:]))

AttributeError: module 'pydrake.multibody.rational_forward_kinematics' has no attribute 'IntegratedRegionSearchOptions'

In [26]:
def launch_integrated_search(region, integrated_search_options, seed_point):
    return cspace_free_region.IntegratedCSpacePolytopeSearch(q_star,filtered_collision_pairs,
                                                      region.A(), region.b(),
                                                     integrated_search_options, 
                                                      solver_options, seed_point
                                                     )
def launch_vector_bisection_search(region, integrated_search_options, seed_point):
    return cspace_free_region.IntegratedCSpacePolytopeSearch(q_star,filtered_collision_pairs,
                                                      region.A(), region.b(),
                                                     integrated_search_options, 
                                                      solver_options, seed_point
                                                     )

In [16]:
ret_vals = []
for tup in search_tuples:
    ret_vals.append(launch_integrated_search(*tup))

NameError: name 'search_tuples' is not defined

In [17]:
i = 1
r = regions[i]
s = seed_points_q[i]
vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
vector_bisection_search_options.max_iters = 1
vector_bisection_search_options.max_feasible_iters = 20
vector_bisection_search_options.search_d = True
vector_bisection_search_options.epsilon_min = FindEpsilonLowerVector(r.A(), r.b(),
                                                         limits_t[0], limits_t[1], seed_points[i,:])
vector_bisection_search_options.epsilon_max = 0.98*vector_bisection_search_options.epsilon_min #np.zeros(r.b().shape[0])

In [18]:
d_final, _ = cspace_free_region.CspacePolytopeBisectionSearchVector(q_star,set(),
                                                      r.A(), r.b(),
                                                     vector_bisection_search_options, 
                                                      solver_options, seed_points_q)

[2022-02-05 15:10:26.860] [console] [info] bilinear alt on d succeeded
[2022-02-05 15:10:26.892] [console] [info] Solver time 2.400635004043579


In [20]:
r_new = HPolyhedron(r.A(), d_final)
if do_viz:
    visualizer.plot_regions([r_new], ellipses=None, region_suffix='_new')

In [17]:
planes = cspace_free_region.separating_planes()

<Expression "(b_constant0 + (t[0] * b_coeff0(0)) + (t[1] * b_coeff0(1)) + (t[2] * b_coeff0(2)))">

In [16]:
#refine with CPP code:
cpp_editted_scalar_regions = []
for i in range(seed_points.shape[0]):
    s = seed_points[i,:]
    region_init = regions[i]
    region_init, region = iris_scalar_handle(s, region_init)
    cpp_editted_scalar_regions.append(region)
    print(f"Completed {i+1}/{seed_points.shape[0]}")

[2022-01-31 15:40:42.251] [console] [info] Solver time 2.6619720458984375
[2022-01-31 15:40:44.826] [console] [info] Solver time 1.0261502265930176


[IRIS +  CERT]: Eps min -0.2652286679481278
[IRIS +  CERT]: Region found, vol =  0.6819787390374958 , halfaxis:  [4.86595118 2.17555524 0.58020292]


[2022-01-31 15:40:48.683] [console] [info] Solver time 1.619724988937378
[2022-01-31 15:40:48.733] [console] [info] epsilon=-0.1326143339740639 is infeasible
[2022-01-31 15:40:53.321] [console] [info] Solver time 1.117055892944336
[2022-01-31 15:40:53.401] [console] [info] epsilon=-0.19892150096109587 is feasible
[2022-01-31 15:40:53.402] [console] [info] reset eps_min=0.0, eps_max=0.06630716698703196


Completed 1/5
[IRIS +  CERT]: Eps min -0.29420562277149487
[IRIS +  CERT]: Region found, vol =  0.5535987103063078 , halfaxis:  [4.69487125 2.58579857 0.62326848]


[2022-01-31 15:41:02.811] [console] [info] Solver time 3.412208080291748
[2022-01-31 15:41:05.195] [console] [info] Solver time 0.7390580177307129
[2022-01-31 15:41:10.948] [console] [info] Solver time 1.3550069332122803
[2022-01-31 15:41:11.041] [console] [info] epsilon=-0.14710281138574743 is feasible
[2022-01-31 15:41:11.042] [console] [info] reset eps_min=0.0, eps_max=0.14710281138574743
[2022-01-31 15:41:16.878] [console] [info] Solver time 2.747184991836548
[2022-01-31 15:41:16.951] [console] [info] epsilon=0.07355140569287372 is infeasible
[2022-01-31 15:41:24.801] [console] [info] Solver time 3.2756950855255127


...

Completed 4/5


[2022-01-31 15:42:31.487] [console] [info] Solver time 4.768555164337158


[IRIS +  CERT]: Eps min -0.33868521538628993
[IRIS +  CERT]: Region found, vol =  0.43617411313252785 , halfaxis:  [3.98248604 1.65632853 1.45588804]


[2022-01-31 15:42:34.253] [console] [info] Solver time 0.7434499263763428
[2022-01-31 15:42:42.592] [console] [info] Solver time 1.8796329498291016
[2022-01-31 15:42:42.727] [console] [info] epsilon=-0.16934260769314496 is feasible
[2022-01-31 15:42:42.728] [console] [info] reset eps_min=0.0, eps_max=0.16934260769314502
[2022-01-31 15:42:52.439] [console] [info] Solver time 2.5359718799591064
[2022-01-31 15:42:52.575] [console] [info] epsilon=0.08467130384657251 is feasible
[2022-01-31 15:42:52.576] [console] [info] reset eps_min=0.0, eps_max=0.08467130384657251


Completed 5/5


In [40]:
if do_viz:
    visualizer.plot_regions(cpp_editted_regions, ellipses=None, region_suffix='cpp_editted_regions')

In [44]:
#setup refine with python
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-4,
    'iris_max_faces': -1
}
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)
regions, ellipses = iris_generator.iris_in_rational_space(seed_points)
iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

snopt_example=[ 0.35244668 -0.19250022 -0.17979872], growth = 1936025801.4851828
snopt_example=[ 0.23144445 -0.04062289 -0.46224621], growth = 2688883094.3830557
snopt_example=[ 0.2001245  -0.10137361 -0.26895992], growth = 1226658660.3889592
snopt_example=[0.         0.         0.22974536], growth = 527829310.38532484
snopt_example=[-0.18114806  0.0938193   0.        ], growth = 416166830.35149866
snopt_example=[-0.1760171   0.09533373  0.        ], growth = 400705370.23392135
snopt_example=[ 0.2157998  -0.11140991 -0.25108731], growth = 1220265554.750688
0
snopt_example=[ 0.16305709  0.71849721 -0.56308786], growth = 2.899642778747243
snopt_example=[-0.60852286 -1.18066191 -0.12769203], growth = 0.6628366692873271
1
Time:   1.61 	Volume:   0.56 	Center: [ 0.18683057  0.06844461 -0.0630242 ]
snopt_example=[ 0.81716573  0.47417491 -0.38295522], growth = 2389296154.6526375
snopt_example=[ 0.77694698  0.54947603 -0.40266231], growth = 1702365874.8724155
snopt_example=[ 0.44441592  0.7602

{<pydrake.geometry.optimization.HPolyhedron at 0x7f9f7db27530>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87f3d0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f9f7f379e70>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87f730>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f9f7da92e70>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87fdf0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f9f7f6fc530>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87fd90>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f9f7dc17570>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87c550>}

In [45]:
if do_viz:
    visualizer.plot_regions(iris_generator.regions, region_suffix = '_python_snopt')

In [ ]:
# refine with python
do_linesearch_cert = True
if do_linesearch_cert:
    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

Starting Region 1/5
min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.56916636 0.77870386 0.56916636 0.56916636 0.77870386 0.56916636
 0.22000147 0.25927221 0.17511844 0.11487268 0.10200084 0.10008813
 0.1746615  0.31397342 0.33108215]
max_eps = [1.13833271 1.55740772 1.13833271 1.13833271 1.55740772 1.13833271
 0.44000293 0.51854441 0.35023687 0.22974536 0.20400167 0.20017626
 0.349323   0.62794684 0.66216431]
Biggest difference: 1.5574077246549023
Smallest difference: 0.20017626488520598
Fixed eps is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.28458318 0.38935193 0.28458318 0.28458318 0.38935193 0.28458318
 0.11000073 0.1296361  0.08755922 0.05743634 0.05100042 0.05004407
 0.08733075 0.15698671 0.16554108]
max_eps = [0.56916636 0.77870386 0.56916636 0.56916636 0.77870386 0.56916636
 0.22000147 0.25927221 0.17511844 0.11487268 0.10200084 0.10008813
 0.1746615  0.31397342 0.33108215]
Biggest difference: 0.7787038623274511
Smallest 

In [ ]:
if do_viz:
    visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')